In [1]:
import pickle
import re
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer


C:\Users\boran\anaconda3\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('articles-pickle/articles_0.pickle', 'rb') as f:
    queries = pickle.load(f)

In [3]:
title = queries[0]["ArticleTitle"]
title = ''.join(title).replace('\n', ' ').strip()

author = queries[0]["Author"]
author = ''.join(author).replace('\n', ' ').strip()

In [4]:
year = queries[0]["Date"]

In [5]:
year = ''.join(year)[:4]

In [6]:
def pdf_to_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    text = text.replace('\n', ' ')
    return text

In [7]:
text = pdf_to_text("C:/Users/boran/Desktop/Local-Citation-Recommendation-main/pdfs/test.pdf")

In [8]:
#text

In [9]:
pattern_to_remove =  r'\b(\d+)\s+([\w\s]+)\((\d{4})\)\.\s*([^\.]+)\.\s*([^,]+),\s*(\d+)\s*\((\d+)\),\s*(\d+)-(\d+)\.\b'
matches = re.finditer(pattern_to_remove, text)

# Initialize a count variable
count = 0

# Iterate through the matches and count them
for match in matches:
    count += 1
    matched_sentence = match.group()
    print(matched_sentence)
    print("\n--------------\n")

# Print the total count of matching sentences
print(f"Total matches found: {count}")

159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

160 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

161 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

162 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

163 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

164 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

165 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye 

In [10]:
text = re.sub(pattern_to_remove, '', text)
#text

In [11]:
def split_article(article_text):
    reference_pattern = r"Kaynakça|References|Bibliography|Citations|Sonnotlar"
    reference_match = re.search(reference_pattern, article_text, re.IGNORECASE)

    if reference_match:
        main_text = article_text[:reference_match.start()] #extract main text from references
        reference_section = article_text[reference_match.start():]

        return main_text, reference_section
    else:
        return article_text, None

text, references = split_article(text)

In [12]:
def window(text, loc, target_str, left_window_size=150, right_window_size=100):
    left_threshold = max(loc - left_window_size, 0)
    right_threshold = min(loc + right_window_size, len(text))

    return text[left_threshold:right_threshold]


def find_references_with_context(text, title, author, year):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    references_with_context = []

    for match in pattern.finditer(text):
        if match.group().lower() != title.lower() and author not in match.group() and str(year) not in match.group():
            match_text = match.group()
            start_loc = text.find(match_text)
            context = window(text, start_loc, match_text, 150, 100)
            references_with_context.append(context)

    return references_with_context

In [13]:
sentences_with_references=find_references_with_context(text, title, author, year)

In [14]:
#sentences_with_references

In [15]:
for i in sentences_with_references:
    print(i)
    print("--------")

y in Turkey and derive  the necessary empirical argument in the theoretical discussion, this article analyses  minutes of the National Security Board (Milli Güvenlik Konseyi) and Consultation  Assembly (Danışma Meclisi) during the relevant discussion
--------
incelenmesi bağlamında aynı saptamayı yapmak zordur; olağanüstü  hal rejimini odağına yerleştiren araştırmaların sayısı ve kapsamı görece sınırlıdır  (Gemalmaz, 1991; Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrud
--------
 Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrudan yöneten askeri rejim, orta vadedeki çabalarını devlet  aklının (raison d’état) çerçevesini çizdiği yeni bir anayasa hazırlatmak üzerine yoğunlaştırmıştır. Yeni ana
--------
ile silahlı mücadelesinde devlet otoritesini  yeniden tesis etmek adına olağanüstü hal rejiminden yararlandığını ve askerlerin  Milli Güvenlik Kurulu (MGK) üzerinden olağanüstü hale karar veren esas yetkili  merci ve siy

In [16]:
len(sentences_with_references)

53

In [17]:
sentences_with_references = list(set(sentences_with_references))

In [18]:
references = queries[0]["References"]

In [19]:
len(references)

55

In [20]:
references

['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
 'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
 'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
 'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
 'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. Journal of Mediterranean and Balkan Intelligence, 3, 25-42.',
 'Bilgiç T Ü (2009). The Military and Europenization Reforms in Turkey. Middle Eastern Studies, 45, 803-824.',
 'Bozaslan H (2014). “Neden Silahlı Mücadele?”: Türkiye Kürdistanı’nda Şiddeti Anlamak. İçinde: Çeğin G ve Şirin İ (der), Türkiye’de Siyasal Şiddetin Boyutları. İstanbul: İletişim Yayınları, 149-164.',
 'Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.',
 'Cizr

In [21]:
sentences_with_references

['al dost-düşman ayrımını devletin  bekası için gerekli gören devlet aklının bir yansımasıdır. Carl Schmitt’in Siyasi  İlahiyat ’ta ileri sürdüğü üzere (2005: 19-20), olağanüstü hal devlet otoritesinin  özünü ortaya koyar. Devlet ‘normal’ hukuku kendin',
 'ygulamalarını şu şekilde  açıklamaktadır: Diyarbakır, Hakkâri, Mardin, Siirt illerinde yaşayan ve bugün 16 yaşında  olan bir genç için sıkıyönetim ve (olağanüstü hal yönetiminin) olağanüstü  bir yönetim biçimi olduğunu kavramak olanaksızdır. Bu gençl',
 'alin, kendini norm yerine yerleştirmesine  kadar varmaktadır. Paye’e göre, olağanüstü halin sürekliliğinde istisnalar kural  olur. Paye’in ifadesiyle (2007: 15), “siyasal örgütlenme şekli olarak olağanüstü  usul, kendini anayasa ve kanunun yerine koy',
 'nin bir kısmında ayrı bir rejim uygulanıyor. Bre taklitçiler, 17 yıl oldu. Siz ülkeyi bölüyorsunuz. ANAP  inşallah akıllanmıştır. Yanımızda yer alır” (Milliyet, 27/06/1995: 15). Erbakan’a göre, olağanüstü hal uygulaması ülkenin bir k

In [22]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

def match_reference(citation, references):
    # Extract year and try to find a part of the author's name
    match = re.search(r'(\d{4})', citation)
    if match:
        year = match.group(1)
        # Split citation and look for potential author name parts
        for part in re.split(r'\W+', citation):
            if part and len(part) > 2 and part.lower() != year:  # Exclude very short parts and the year
                for ref in references:
                    if year in ref and part in ref:
                        return ref
    return "No matching reference found"

In [23]:
for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        matched_reference = match_reference(citation, references)
        print(f"Window: {window}")
        print(f"Citation: {citation}")
        print(f"Matched Reference: {matched_reference}\n")

Window: al dost-düşman ayrımını devletin  bekası için gerekli gören devlet aklının bir yansımasıdır. Carl Schmitt’in Siyasi  İlahiyat ’ta ileri sürdüğü üzere (2005: 19-20), olağanüstü hal devlet otoritesinin  özünü ortaya koyar. Devlet ‘normal’ hukuku kendin
Citation: (2005: 19-20)
Matched Reference: No matching reference found

Window: alin, kendini norm yerine yerleştirmesine  kadar varmaktadır. Paye’e göre, olağanüstü halin sürekliliğinde istisnalar kural  olur. Paye’in ifadesiyle (2007: 15), “siyasal örgütlenme şekli olarak olağanüstü  usul, kendini anayasa ve kanunun yerine koy
Citation: (2007: 15)
Matched Reference: No matching reference found

Window: nin bir kısmında ayrı bir rejim uygulanıyor. Bre taklitçiler, 17 yıl oldu. Siz ülkeyi bölüyorsunuz. ANAP  inşallah akıllanmıştır. Yanımızda yer alır” (Milliyet, 27/06/1995: 15). Erbakan’a göre, olağanüstü hal uygulaması ülkenin bir kısmında farklı bi
Citation: (Milliyet, 27/06/1995: 15)
Matched Reference: Milliyet (27/06/1995).

Wi

In [24]:
citations_2 = []

for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        citations_2.append(citation)

In [25]:
citations_2

['(2005: 19-20)',
 '(2007: 15)',
 '(Milliyet, 27/06/1995: 15)',
 '(Özcan, 2007: 43)',
 '(2000: 95)',
 '(Danışma Meclisi, 1983: 6)',
 '(Milliyet, 25/01/1997: 7)',
 '(2005: 10- 12)',
 '(2006: 194-204)',
 '(Öztan, 2014: 192)',
 '(Üskül, 1988: 103)',
 '(1984: 2)',
 '(Danışma Meclisi, 1983: 7)',
 '(Yüksekova Haber, 14/08/2011)',
 '(2005: 10- 12)',
 '(Milliyet, 03/10/1984: 6)',
 '(Danışma Meclisi, 1983: 4)',
 '(1984)',
 '(Milliyet,  25/08/1984: 6)',
 '(2014: 221)',
 '(Bağımsızlık) isimli derginin  Ağustos 1984 tarihli sayısında  (1984: 2)',
 '(2000: 116)',
 '(Danışma Meclisi, 1983: 3)',
 '(1999: 31)',
 '(07/10/1984: 7)',
 '(Bozdemir,  1982: 163)',
 '(İnsel, 1997:  15-19)',
 '(Danışma Meclisi, 1983:  33)',
 '(Bezci, 2014: 29-34)',
 '(1961: 39-54)',
 '(Cizre, 1999: 44)',
 '(Cizre, 1993: 277)',
 '(2009: 224)',
 '(bu tür kararların kaldırılabilmesi için yeniden sıkıyönetim  ilanını istemek gerekiyor!) (1988: 89)',
 '(2014: 221)',
 '(2004: 143)',
 '(Gemalmaz, 1991; Üskül, 1988; Duran, 1995)',
 '(

In [26]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

In [27]:
def calculate_cosine_similarity_tfidf(citations, references):
    # Combine both lists of strings
    all_strings = citations + references

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Transform the strings into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_strings)

    # Calculate cosine similarity between each pair of strings
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Iterate through citations and find the best match in references
    for i, citation in enumerate(citations):
        # Find the index of the most similar reference
        best_match_index = similarity_matrix[i, len(citations):].argmax()

        # Get the corresponding reference string
        best_match_reference = references[best_match_index]

        # Get the cosine similarity score
        similarity_score = similarity_matrix[i, len(citations) + best_match_index]

        # Print the results
        if similarity_score > 0.25:
            print(f"Citation: {citation}")
            print(f"Reference: {best_match_reference}")
            print(f"Cosine Similarity Score: {similarity_score:.4f}")
            print("-" * 30)

calculate_cosine_similarity_tfidf(citations_2, references)


Citation: (Milliyet, 27/06/1995: 15)
Reference: Milliyet (27/06/1995).
Cosine Similarity Score: 0.8938
------------------------------
Citation: (Danışma Meclisi, 1983: 6)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Cosine Similarity Score: 0.6482
------------------------------
Citation: (Milliyet, 25/01/1997: 7)
Reference: Milliyet (25/01/1997).
Cosine Similarity Score: 1.0000
------------------------------
Citation: (2006: 194-204)
Reference: Neocleous M (2006). The Problem with Normality: Taking Exception to ‘Permanent Emergency’. Alternatives, 31, 194-204.
Cosine Similarity Score: 0.4152
------------------------------
Citation: (Öztan, 2014: 192)
Reference: Öztan G G (2014). Türkiye’de Militarizm: Zihniyet, Pratik ve Progadanda. İstanbul: Bilgi Üniversitesi Yayınları.
Cosine Similarity Score: 0.2703
------------------------------
Citation: (Üskül, 1988: 103)
Reference: Üskül Z (1988). Türkiye

In [28]:
def preprocess(citations_2):
    # Lowercasing and tokenization for each sentence
    nltk.download('punkt')
    tokenized_citations = [nltk.word_tokenize(sentence.lower()) for sentence in citations_2]

    # Removing Punctuation and Stopword Removal for each sentence
    nltk.download('stopwords')
    stop_words = set(stopwords.words('turkish'))
    preprocessed_citations = []

    for tokens in tokenized_citations:
        tokens = [word for word in tokens if word.isalnum()]
        filtered_tokens = [word for word in tokens if word not in stop_words]
        preprocessed_citations.append(filtered_tokens)

    # Lemmatization or Stemming (Using SnowballStemmer) for each sentence
    stemmer=TurkishStemmer()
    stemmed_citations = []

    for tokens in preprocessed_citations:
        stemmed_tokens = [stemmer.stemWord(word) for word in tokens]
        stemmed_citations.append(stemmed_tokens)

    # Print the preprocessed sentences
    for sentence_tokens in stemmed_citations:
        preprocessed_citations = ' '.join(sentence_tokens)

    print(preprocessed_citations)
    
    return preprocessed_citations

In [29]:
def create_embeddings_and_model(citations, references, model_name = 'all-mpnet-base-v2'):
    
    model = SentenceTransformer(model_name, device='cuda')
    
    citation_embeddings = model.encode(citations, convert_to_tensor=True)
    reference_embeddings = model.encode(references, convert_to_tensor=True)
    
    return model, citation_embeddings, reference_embeddings
    

In [30]:
model, citation_embeddings, reference_embeddings = create_embeddings_and_model(citations_2, references, model_name='multi-qa-mpnet-base-dot-v1')

In [31]:
def find_matching_citations_references(citation_embeddings, reference_embeddings, citations, references):
    # Calculate cosine similarity between citation and reference embeddings
    similarity_matrix = cosine_similarity(citation_embeddings.cpu().numpy(), reference_embeddings.cpu().numpy())

    # Find the indices of the maximum similarity values for each citation
    best_matches = similarity_matrix.argmax(axis=1)

    # Print matched citations and references with their cosine similarities
    for i, (citation, reference_idx) in enumerate(zip(citations, best_matches)):
        matched_reference = references[reference_idx]
        print(f"Match {i + 1}:")
        print(f"Citation: {citation}")
        print(f"Reference: {matched_reference}")
        print("="*30)

In [32]:
find_matching_citations_references(citation_embeddings, reference_embeddings, citations_2, references)

Match 1:
Citation: (2005: 19-20)
Reference: Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.
Match 2:
Citation: (2007: 15)
Reference: Serxwebun (10/1993).
Match 3:
Citation: (Milliyet, 27/06/1995: 15)
Reference: Milliyet (30/07/1996)
Match 4:
Citation: (Özcan, 2007: 43)
Reference: Soysal İ (1992). Kaldırılmalı mı?. Milliyet, 4/3/1992.
Match 5:
Citation: (2000: 95)
Reference: Milliyet (30/07/1996)
Match 6:
Citation: (Danışma Meclisi, 1983: 6)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Match 7:
Citation: (Milliyet, 25/01/1997: 7)
Reference: Milliyet (30/07/1996)
Match 8:
Citation: (2005: 10- 12)
Reference: Serxwebun (10/1993).
Match 9:
Citation: (2006: 194-204)
Reference: Milliyet (30/07/1996)
Match 10:
Citation: (Öztan, 2014: 192)
Reference: Soysal İ (1992). Kaldırılmalı mı?. Milliyet, 4/3/1992.
Match 11:
Citation: (Üskül, 1988: 103)
Reference: İns

In [33]:
import yake

In [34]:
def extract_and_concatenate_with_parentheses(text, min_length=5):
    # Use regular expression to find substrings within parentheses
    matches = re.findall(r'\(.*?\)', text)
    # Concatenate the substrings found within parentheses with parentheses
    result_string = ' '.join(match for match in matches if len(match) > min_length) 
    print(result_string)
    return result_string

In [35]:
input_Str = extract_and_concatenate_with_parentheses(text)

(Milli Güvenlik Konseyi) (Danışma Meclisi) (Cizre, 1999;  Dağı, 1996;  Kaplan, 2002; Narlı, 2000; Demirel, 2005; Heper ve Güney, 1996;  Brown, 1989) (Gemalmaz, 1991; Üskül, 1988; Duran, 1995) (raison d’état) (Sancar, 2012: 24-25) (2009: 224) (Agamben, 1998: 165) (2007: 15) (2005: 10- 12) (2006: 194-204) (Danışma Meclisi, 1983: 4) (Danışma Meclisi, 1983: 3) (Danışma Meclisi, 1983: 3) (Danışma Meclisi, 1983: 5) (Danışma Meclisi, 1983: 6) (Danışma Meclisi, 1983: 7) (...hukuk dışı tutum ve davranışlarıyla meşruluğunu  kaybetmiş bir iktidar...) (...direnme hakkını...) (Danışma Meclisi, 1983: 30) (Danışma Meclisi, 1983:  33) (1999: 70) (1961: 39-54) (Bozdemir,  1982: 163) (2000: 95) (ANAP) (1999: 31) (Cizre, 1999: 44) (Cizre, 1993: 277) (1984) (2004: 143) (Yüksekova Haber, 14/08/2011) (2014: 158) (2014: 221) (Bağımsızlık) (1984: 2) (Ağustos 1984: 3) (olağanüstü hal yönetiminin) (1988: 85) (Öztan, 2014: 192) (Üskül, 1988: 103) (bu tür kararların kaldırılabilmesi için yeniden sıkıyönetim  ilan

In [36]:
input_Str

'(Milli Güvenlik Konseyi) (Danışma Meclisi) (Cizre, 1999;  Dağı, 1996;  Kaplan, 2002; Narlı, 2000; Demirel, 2005; Heper ve Güney, 1996;  Brown, 1989) (Gemalmaz, 1991; Üskül, 1988; Duran, 1995) (raison d’état) (Sancar, 2012: 24-25) (2009: 224) (Agamben, 1998: 165) (2007: 15) (2005: 10- 12) (2006: 194-204) (Danışma Meclisi, 1983: 4) (Danışma Meclisi, 1983: 3) (Danışma Meclisi, 1983: 3) (Danışma Meclisi, 1983: 5) (Danışma Meclisi, 1983: 6) (Danışma Meclisi, 1983: 7) (...hukuk dışı tutum ve davranışlarıyla meşruluğunu  kaybetmiş bir iktidar...) (...direnme hakkını...) (Danışma Meclisi, 1983: 30) (Danışma Meclisi, 1983:  33) (1999: 70) (1961: 39-54) (Bozdemir,  1982: 163) (2000: 95) (ANAP) (1999: 31) (Cizre, 1999: 44) (Cizre, 1993: 277) (1984) (2004: 143) (Yüksekova Haber, 14/08/2011) (2014: 158) (2014: 221) (Bağımsızlık) (1984: 2) (Ağustos 1984: 3) (olağanüstü hal yönetiminin) (1988: 85) (Öztan, 2014: 192) (Üskül, 1988: 103) (bu tür kararların kaldırılabilmesi için yeniden sıkıyönetim  ila

In [37]:
language = "tr"
max_ngram_size = 5
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 6
numOfKeywords = 40

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(input_Str)

for kw in keywords:
    print(kw)

('Milli Güvenlik Konseyi', 0.0003097728850107684)
('Danışma Meclisi', 0.00038431166589923725)
('hukuk dışı tutum ve davranışlarıyla', 0.0016161705602431126)
('davranışlarıyla meşruluğunu kaybetmiş bir iktidar', 0.0016161705602431126)
('dışı tutum ve davranışlarıyla meşruluğunu', 0.0019132668352714773)
('tutum ve davranışlarıyla meşruluğunu kaybetmiş', 0.0019132668352714773)
('Heper ve Güney', 0.004213405673414221)
('Milli Güvenlik', 0.0047424302747162295)
('Güvenlik Konseyi', 0.0047424302747162295)
('Danışma', 0.006523351120660553)
('Meclisi', 0.006523351120660553)
('hukuk dışı tutum', 0.007452183718451643)
('meşruluğunu kaybetmiş bir iktidar', 0.007452183718451643)
('dışı tutum ve davranışlarıyla', 0.008794328895866151)
('tutum ve davranışlarıyla meşruluğunu', 0.008794328895866151)
('davranışlarıyla meşruluğunu kaybetmiş', 0.008794328895866151)
('Milliyet', 0.01994213168312439)
('raison d’état', 0.029903988823314066)
('Cizre', 0.034003112588244115)
('hukuk dışı', 0.03511036368486145)


In [38]:
def extract_and_filter_matches(text):
    # Use regular expression to find substrings within parentheses
    # Find matches within parentheses, including the brackets
    matches_parentheses = re.findall(r'(\(.*?\))', text)

    # Find matches within square brackets, including the brackets
    matches_square_brackets = re.findall(r'(\[.*?\])', text)
    
    filtered_matches_2 = [match for match in matches_parentheses if (':' in match and ',' in match) or ':' in match or ';' in match or match.isdigit()]
    filtered_matches_3 = [match for match in matches_square_brackets if match[1:-1].isdigit()]

    return  filtered_matches_2 + filtered_matches_3

In [39]:
filtered_matches_2 = extract_and_filter_matches(input_Str)

In [40]:
print(filtered_matches_2)

['(Cizre, 1999;  Dağı, 1996;  Kaplan, 2002; Narlı, 2000; Demirel, 2005; Heper ve Güney, 1996;  Brown, 1989)', '(Gemalmaz, 1991; Üskül, 1988; Duran, 1995)', '(Sancar, 2012: 24-25)', '(2009: 224)', '(Agamben, 1998: 165)', '(2007: 15)', '(2005: 10- 12)', '(2006: 194-204)', '(Danışma Meclisi, 1983: 4)', '(Danışma Meclisi, 1983: 3)', '(Danışma Meclisi, 1983: 3)', '(Danışma Meclisi, 1983: 5)', '(Danışma Meclisi, 1983: 6)', '(Danışma Meclisi, 1983: 7)', '(Danışma Meclisi, 1983: 30)', '(Danışma Meclisi, 1983:  33)', '(1999: 70)', '(1961: 39-54)', '(Bozdemir,  1982: 163)', '(2000: 95)', '(1999: 31)', '(Cizre, 1999: 44)', '(Cizre, 1993: 277)', '(2004: 143)', '(2014: 158)', '(2014: 221)', '(1984: 2)', '(Ağustos 1984: 3)', '(1988: 85)', '(Öztan, 2014: 192)', '(Üskül, 1988: 103)', '(1988: 89)', '(2010: 409-410)', '(07/10/1984: 7)', '(Milliyet, 03/10/1984: 6)', '(Milliyet,  25/08/1984: 6)', '(2005: 19-20)', '(2005: 647)', '(Bezci, 2014: 29-34)', '(Harp Akademileri Komutanlığı, 1993: 20)', '(Milliyet

In [41]:
import os

In [ ]:
pdf_folder_path = r'C:\Users\boran\Desktop\Local-Citation-Recommendation-main\pdfs'


def process_pdfs_in_folder(folder_path):
    pdf_texts = {}

    # List all PDF files in the folder
    pdf_files = [file for file in os.listdir(folder_path) if file.lower().endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_file_path = os.path.join(folder_path, pdf_file)

        # Apply pdf_to_text function to each PDF file
        text = pdf_to_text(pdf_file_path)
        pattern_to_remove =  r'\b(\d+)\s+([\w\s]+)\((\d{4})\)\.\s*([^\.]+)\.\s*([^,]+),\s*(\d+)\s*\((\d+)\),\s*(\d+)-(\d+)\.\b'
        text = re.sub(pattern_to_remove, '', text)
        text, references = split_article(text)
        # Add the result to the dictionary with the PDF file name as the key
        pdf_texts[pdf_file] = text

    return pdf_texts

# Example usage
pdf_texts = process_pdfs_in_folder(pdf_folder_path)


In [ ]:
pdf_with_citations = {}

In [ ]:
for key in pdf_texts.keys():
    pdf_with_citations[key] = extract_and_filter_matches(pdf_texts[key])
    

In [ ]:
for key, value in pdf_with_citations.items():
    print(key)
    for item in value:
        print(f'  - {item}')

In [ ]:
example = "Merhaba benim adım Boran Kahraman[1a23]"

In [ ]:
extract_and_filter_matches(example)